In [1]:
import warnings
from tabletex import table_tex
from matplotlib.pyplot import cla, xlabel
from sklearn.manifold import TSNE
import seaborn as sns
from tqdm import tqdm

from utils import *
from classes import *
from simulation import *
import streamlit as st
import numpy as np
import pickle as pkl
import sys
# sys.path.append('./')
#import livejson
import json
#import SessionState
# ignore warnings
warnings.filterwarnings("ignore")


param_sim = {
    'n_features': 25,
    'n_classes': 2,
    'n_samples': 1000,
    'wd_para': 2.,
    'beta': [0.1, 0.1, 0.3],
    'coef_tt': 1.1,
    'rho': 0.1,
    'path_data': './dataclassif.csv'
}

params_classifcaus = {
    "encoded_features": 25,
    "alpha_wass": 0.01,
    "batch_size": 128,
    "epochs": 30,
    "lr": 0.001,
    "patience": 7,
}

idx = np.arange(param_sim['n_features'])
param_sim['beta'] = (-1) ** idx * np.exp(-idx / 20.)

# class for Experiment: for evry wd_para simulate new data and luanch benchmark


class Experiment:
    def __init__(self, param_sim, params_classifcaus, list_wd_param, num_exp, list_models, mode="onemodel"):
        self.param_sim = param_sim
        self.params_classifcaus = params_classifcaus
        self.list_wd_param = list_wd_param
        self.num_exp = num_exp
        self.list_models = list_models
        self.mode = mode
        self.d_all_exp = {}

    def save_results_exp(self, i_exp):
        d = {'i_exp': i_exp, 'list_wd_param': self.list_wd_param, 'list_models': self.list_models, 'mode': self.mode,
             'num_exp': self.num_exp, 'param_sim': self.param_sim, 'params_classifcaus': self.params_classifcaus}
        self.d_all_exp[i_exp] = d

    def run(self):
        d = {}
        d_latex = []
        for wd_para in self.list_wd_param:
            d_sim = {}
            for exp in tqdm(range(self.num_exp)):
                #print(f'wd_para: {wd_para}, exp: {exp}')
                # print('Simulation')
                self.param_sim['wd_para'] = wd_para
                self.sim = Simulation(self.param_sim)
                self.sim.simule()
                # get param
                d_sim['wd_para'] = wd_para
                d_sim['wd'] = self.sim.wd
                d_sim['y_0_perc'] = self.sim.y_0_perc
                d_sim['y_1_perc'] = self.sim.y_1_perc

                # Bnechmark
                # print('Benchmark')
                Bench = BenchmarkClassif(
                    self.params_classifcaus, self.list_models)

                df_results, dic_fig, d_all = Bench.evall_all_bench(
                    mode=self.mode)

                #print('End of Benchmark')
                df_results = df_results.drop(['f1_0', 'f1_1', 'mae'], axis=1)
                # rename columns
                df_results.columns.values[-3] = 'DTV_1'
                df_results.columns.values[-4] = 'DTV_0'
                # round values to 3 decimals
                df_results = df_results.round(3)
                print(df_results)
                d[f'wd_para_{wd_para}_exp_{exp}'] = {'df': df_results, 'wd_param': wd_para, 'exp': exp,
                                                     'wd': self.sim.wd, 'y_0_perc': self.sim.y_0_perc, 'y_1_perc': self.sim.y_1_perc}

                print({'wd_param': wd_para,
                       'wd': self.sim.wd, 'y_0_perc': self.sim.y_0_perc, 'y_1_perc': self.sim.y_1_perc})
                # display results
                # print(df_results)
                # save results as csv
                # to latex 
                d_latex.append(table_tex(df = df_results, title=f'wd_para_{wd_para}_wd_{self.sim.wd}') )
                                         
                fig_0 = boxplot_F(
                    list_models, d_all, ylabel='diff proba 0', option=0)
                plt.savefig(f"boxplot_F_{exp}_{mode}.png")
                fig_1 = boxplot_F(
                    list_models, d_all, ylabel='diff proba 1', option=1)
                plt.savefig(f"boxplot_F_{exp}_{mode}.png")
                plt.close('all')

                fig_dist_0_list = dist_F(
                    list_models, d_all, ylabel='', option=0)
                fig_dist_1_list = dist_F(
                    list_models, d_all, ylabel='', option=1)
                plt.close('all')
                self.save_results_exp(exp)

        ##print(f'd_sim: {d_sim}')
        ##print(f'd_all_exp: {self.d_all_exp}')
        # print(d)
        self.d = d
        self.d_latex = d_latex


if __name__ == "__main__":

    list_models = ["ClassCaus", "lgbm", "xgb"]

    num_exp = 3
    mode = "onemodel"
    # get list of wd_param
    list_wd_param = [0., 2., 4., 6.]
    #np.arange(0.1, 5., 0.5)
    # create object
    exp = Experiment(param_sim, params_classifcaus,
                     list_wd_param, num_exp, list_models, mode=mode)
    # run
    exp.run()
    print(exp.d)
    print(exp.d_latex)
    # save as pickle
    """with open('results_exp.pkl', 'wb') as fp:
        pkl.dump(exp.d, fp)"""


100%|██████████| 9/9 [00:00<00:00, 24401.25it/s]


1
2
3
4
5
6
7
8
9


  0%|          | 0/3 [00:00<?, ?it/s]

WD =  24.087810516357422
tt = 1 : 46 % 
Y_0 = 1 : 47 % 
Y_1 = 1 : 63 % 
KLD = 0.10335976631805187
Memory usage of dataframe is 0.26 MB
Memory usage after optimization is: 0.06 MB
Decreased by 76.8%
Evaluation for model ClassCaus
0:	[0s / 0s],		train_loss: 0.6921,	train_loss_classif: 0.6916,	train_loss_wass: 0.0530,	val_loss: 0.6943,	val_loss_classif: 0.6939,	val_loss_wass: 0.0451
1:	[0s / 0s],		train_loss: 0.6880,	train_loss_classif: 0.6876,	train_loss_wass: 0.0400,	val_loss: 0.6970,	val_loss_classif: 0.6966,	val_loss_wass: 0.0426
2:	[0s / 0s],		train_loss: 0.6820,	train_loss_classif: 0.6817,	train_loss_wass: 0.0354,	val_loss: 0.6980,	val_loss_classif: 0.6976,	val_loss_wass: 0.0406
3:	[0s / 0s],		train_loss: 0.6793,	train_loss_classif: 0.6790,	train_loss_wass: 0.0366,	val_loss: 0.6991,	val_loss_classif: 0.6988,	val_loss_wass: 0.0384
4:	[0s / 0s],		train_loss: 0.6731,	train_loss_classif: 0.6727,	train_loss_wass: 0.0326,	val_loss: 0.6888,	val_loss_classif: 0.6884,	val_loss_wass: 0.0345
5

 33%|███▎      | 1/3 [00:15<00:30, 15.17s/it]

...

WD =  273.0181579589844
tt = 1 : 51 % 
Y_0 = 1 : 46 % 
Y_1 = 1 : 61 % 
KLD = 0.10213685261278688
Memory usage of dataframe is 0.26 MB
Memory usage after optimization is: 0.06 MB
Decreased by 76.8%
Evaluation for model ClassCaus
0:	[0s / 0s],		train_loss: 0.6976,	train_loss_classif: 0.6955,	train_loss_wass: 0.2101,	val_loss: 0.6905,	val_loss_classif: 0.6895,	val_loss_wass: 0.1022
1:	[0s / 1s],		train_loss: 0.6865,	train_loss_classif: 0.6853,	train_loss_wass: 0.1232,	val_loss: 0.6800,	val_loss_classif: 0.6781,	val_loss_wass: 0.1894
2:	[0s / 2s],		train_loss: 0.6713,	train_loss_classif: 0.6686,	train_loss_wass: 0.2686,	val_loss: 0.6617,	val_loss_classif: 0.6576,	val_loss_wass: 0.4122
3:	[1s / 3s],		train_loss: 0.6500,	train_loss_classif: 0.6436,	train_loss_wass: 0.6409,	val_loss: 0.6342,	val_loss_classif: 0.6296,	val_loss_wass: 0.4536
4:	[1s / 4s],		train_loss: 0.6343,	train_loss_classif: 0.6244,	train_loss_wass: 0.9899,	val_loss: 0.6248,	val_loss_classif: 0.6208,	val_loss_wass: 0.4064
5:

 67%|██████▋   | 2/3 [02:03<01:01, 61.73s/it]

WD =  258.3604736328125
tt = 1 : 50 % 
Y_0 = 1 : 49 % 
Y_1 = 1 : 62 % 
KLD = 0.10057736863301482
Memory usage of dataframe is 0.26 MB
Memory usage after optimization is: 0.06 MB
Decreased by 76.8%
Evaluation for model ClassCaus
0:	[0s / 0s],		train_loss: 0.6932,	train_loss_classif: 0.6895,	train_loss_wass: 0.3707,	val_loss: 0.6843,	val_loss_classif: 0.6819,	val_loss_wass: 0.2460
1:	[0s / 1s],		train_loss: 0.6865,	train_loss_classif: 0.6852,	train_loss_wass: 0.1277,	val_loss: 0.6799,	val_loss_classif: 0.6782,	val_loss_wass: 0.1665
2:	[0s / 1s],		train_loss: 0.6822,	train_loss_classif: 0.6815,	train_loss_wass: 0.0765,	val_loss: 0.6749,	val_loss_classif: 0.6733,	val_loss_wass: 0.1575
3:	[0s / 2s],		train_loss: 0.6770,	train_loss_classif: 0.6759,	train_loss_wass: 0.1047,	val_loss: 0.6712,	val_loss_classif: 0.6693,	val_loss_wass: 0.1961
4:	[0s / 3s],		train_loss: 0.6663,	train_loss_classif: 0.6651,	train_loss_wass: 0.1231,	val_loss: 0.6658,	val_loss_classif: 0.6637,	val_loss_wass: 0.2154
5:

100%|██████████| 3/3 [03:08<00:00, 62.77s/it]

{'wd_para_0.0_exp_0': {'df':            acc_0  acc_1  auc_0  auc_1   pehe  DTV_0  DTV_1  std_diff_0  \
ClassCaus  0.766  0.778  0.851  0.886  0.117  0.139  0.133       0.178   
lgbm       0.812  0.838  0.798  0.818  0.136  0.178  0.130       0.247   
xgb        0.812  0.838  0.811  0.823  0.130  0.166  0.129       0.247   

           std_diff_1  
ClassCaus       0.193  
lgbm            0.237  
xgb             0.240  , 'wd_param': 0.0, 'exp': 0, 'wd': 24.087810516357422, 'y_0_perc': 47, 'y_1_perc': 63}, 'wd_para_0.0_exp_1': {'df':            acc_0  acc_1  auc_0  auc_1   pehe  DTV_0  DTV_1  std_diff_0  \
ClassCaus  0.819  0.800  0.911  0.888  0.095  0.105  0.076       0.159   
lgbm       0.862  0.828  0.848  0.845  0.134  0.176  0.145       0.243   
xgb        0.862  0.828  0.856  0.837  0.131  0.176  0.159       0.250   

           std_diff_1  
ClassCaus       0.145  
lgbm            0.233  
xgb             0.244  , 'wd_param': 0.0, 'exp': 1, 'wd': 24.286033630371094, 'y_0_perc': 47, 